In [17]:
import pandas as pd
from pathlib import Path
from tqdm import tqdm
data_dir = Path("/Users/othmaneechchabi/Desktop/College/Grad/Career/UM6P/fert-recon-map/public/data/fertimap_grid.csv")
grid_df = pd.read_csv(data_dir)
grid_df.columns

Index(['lat', 'lon', 'region', 'province', 'province_id', 'commune',
       'soil_type', 'texture', 'ph', 'organic_matter_pct', 'p2o5_mgkg',
       'k2o_mgkg', 'target_crop_id', 'target_crop_name', 'target_yield_min',
       'target_yield_max', 'target_yield_step', 'target_yield_unit',
       'target_expected_yield', 'scenario_expected_yield', 'rec_n_kg_ha',
       'rec_p_kg_ha', 'rec_k_kg_ha', 'rec_cost_dh_ha',
       'regional_recommendations', 'generic_recommendations'],
      dtype='object')

In [18]:
grid_df

,lat,lon,region,province,province_id,commune,soil_type,texture,ph,organic_matter_pct,...,target_yield_step,target_yield_unit,target_expected_yield,scenario_expected_yield,rec_n_kg_ha,rec_p_kg_ha,rec_k_kg_ha,rec_cost_dh_ha,regional_recommendations,generic_recommendations
0,29.775000,-5.741667,NaN,NaN,19.0,M\'Hamid El Ghizlane,Texture globale,globale,7.95,0.87,...,5.0,qx/ha,40.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN
1,29.775000,-5.741667,NaN,NaN,19.0,M\'Hamid El Ghizlane,Texture globale,globale,7.95,0.87,...,5.0,qx/ha,40.0,50.0,161.69,82.17,165.1,2845.81,NaN,NaN
2,29.775000,-5.741667,NaN,NaN,19.0,M\'Hamid El Ghizlane,Texture globale,globale,7.95,0.87,...,5.0,qx/ha,40.0,60.0,NaN,NaN,NaN,NaN,NaN,NaN
3,29.775000,-5.733333,NaN,NaN,19.0,M\'Hamid El Ghizlane,Texture globale,globale,7.95,0.87,...,5.0,qx/ha,40.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN
4,29.775000,-5.733333,NaN,NaN,19.0,M\'Hamid El Ghizlane,Texture globale,globale,7.95,0.87,...,5.0,qx/ha,40.0,50.0,161.69,82.17,165.1,2845.81,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616765,35.933333,-5.391667,NaN,NaN,NaN,NaN,Texture globale,globale,7.22,13.90,...,5.0,qx/ha,40.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN
616766,35.933333,-5.391667,NaN,NaN,NaN,NaN,Texture globale,globale,7.22,13.90,...,5.0,qx/ha,40.0,60.0,NaN,NaN,NaN,NaN,NaN,NaN
616767,35.933333,-5.383333,NaN,NaN,NaN,NaN,Texture globale,globale,7.22,13.90,...,5.0,qx/ha,40.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN
616768,35.933333,-5.383333,NaN,NaN,NaN,NaN,Texture globale,globale,7.22,13.90,...,5.0,qx/ha,40.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
regional_count = grid_df["regional_recommendations"].notna().sum()
generic_count = grid_df["generic_recommendations"].notna().sum()
both_count = (grid_df["regional_recommendations"].notna() & grid_df["generic_recommendations"].notna()).sum()

print("regional_recommendations non-null:", regional_count)
print("generic_recommendations non-null:", generic_count)
print("both non-null:", both_count)

regional_recommendations non-null: 387200
generic_recommendations non-null: 0
both non-null: 0


In [20]:
# print(grid_df[grid_df["regional_recommendations"].notna()==True]["regional_recommendations"].iloc[60])

### Parsing recommendation paragraphs

The `regional_recommendations` column mixes fertilizer instructions from multiple sections in a single paragraph. The next cells define a parser that normalizes those blocks into structured quantities we can reuse elsewhere in the notebook.

In [21]:
import re

FERTILIZER_LINE_PATTERN = re.compile(
    r"(?P<qty>\d+(?:\.\d+)?)\s*qx/ha\s+d(?:u|['’])?\s*(?P<product>[A-Za-z0-9().]+)\s+comme engrais de\s+(?P<mode>fond|couverture)",
    re.IGNORECASE,
)

COST_PATTERN = re.compile(
    r"pour un cout de\s*(?P<cost>\d+(?:\.\d+)?)\s*dh/ha",
    re.IGNORECASE,
)

SECTION_MARKERS = [
    ("regional_formula", "Recommandations basées sur la formule régionale"),
    ("selected_yield", "Les recommandations pour le rendement"),
    ("generic", "Recommandations basées sur les formules"),
]

def fix_encoding(text: str) -> str:
    if not isinstance(text, str):
        return ""
    try:
        return text.encode("latin1").decode("utf-8")
    except (UnicodeEncodeError, UnicodeDecodeError):
        return text

def parse_recommendation_block(text: str) -> dict:
    parsed = {
        "regional_formula": [],
        "selected_yield": [],
        "generic": [],
        "cost_dh_ha": None,
    }
    if not isinstance(text, str):
        return parsed

    text = fix_encoding(text)
    section = None

    for raw_line in text.splitlines():
        line = raw_line.strip()
        if not line:
            continue

        lower_line = line.lower()
        matched_marker = False
        for label, marker in SECTION_MARKERS:
            if lower_line.startswith(marker.lower()):
                section = label
                matched_marker = True
                break
        if matched_marker:
            continue

        clean_line = line.replace('(*)', '').strip()
        cost_match = COST_PATTERN.search(clean_line)
        if cost_match:
            parsed["cost_dh_ha"] = float(cost_match.group("cost"))
            continue

        fert_match = FERTILIZER_LINE_PATTERN.search(clean_line)
        if fert_match and section:
            parsed[section].append({
                "qty_qx_ha": float(fert_match.group("qty")),
                "product": fert_match.group("product"),
                "mode": fert_match.group("mode").lower(),
            })

    return parsed

In [22]:
non_null_mask = grid_df["regional_recommendations"].notna()
parsed_recos = grid_df.loc[non_null_mask, "regional_recommendations"].apply(parse_recommendation_block)

max_generic_entries = parsed_recos.apply(lambda block: len(block["generic"])).max()

def first_entry(entries, mode=None):
    for entry in entries:
        if mode is None or entry["mode"] == mode:
            return entry
    return None

def to_wide(parsed_block):
    row = {}
    regional_entry = first_entry(parsed_block["regional_formula"])
    row["regional_fond_qty_qx_ha"] = regional_entry["qty_qx_ha"] if regional_entry else None
    row["regional_fond_product"] = regional_entry["product"] if regional_entry else None

    selected_fond = first_entry(parsed_block["selected_yield"], mode="fond")
    row["selected_fond_qty_qx_ha"] = selected_fond["qty_qx_ha"] if selected_fond else None
    row["selected_fond_product"] = selected_fond["product"] if selected_fond else None

    selected_couverture = first_entry(parsed_block["selected_yield"], mode="couverture")
    row["selected_couverture_qty_qx_ha"] = selected_couverture["qty_qx_ha"] if selected_couverture else None
    row["selected_couverture_product"] = selected_couverture["product"] if selected_couverture else None

    for idx in range(max_generic_entries):
        entry = parsed_block["generic"][idx] if idx < len(parsed_block["generic"]) else None
        row[f"generic_entry_{idx + 1}_product"] = entry["product"] if entry else None
        row[f"generic_entry_{idx + 1}_mode"] = entry["mode"] if entry else None
        row[f"generic_entry_{idx + 1}_qty_qx_ha"] = entry["qty_qx_ha"] if entry else None

    row["selected_entry_count"] = len(parsed_block["selected_yield"])
    row["generic_entry_count"] = len(parsed_block["generic"])
    row["reco_cost_dh_ha"] = parsed_block["cost_dh_ha"]
    return row

parsed_structured = pd.DataFrame([to_wide(block) for block in tqdm(parsed_recos)], index=parsed_recos.index)

grid_df = grid_df.join(parsed_structured)

display_columns = [
    "regional_fond_product",
    "regional_fond_qty_qx_ha",
    "selected_fond_product",
    "selected_fond_qty_qx_ha",
    "selected_couverture_product",
    "selected_couverture_qty_qx_ha",
]

for idx in range(1, max_generic_entries + 1):
    display_columns.extend([
        f"generic_entry_{idx}_product",
        f"generic_entry_{idx}_mode",
        f"generic_entry_{idx}_qty_qx_ha",
    ])

display_columns.append("reco_cost_dh_ha")

grid_df.loc[non_null_mask, display_columns].head()

100%|██████████| 387200/387200 [00:00<00:00, 427440.61it/s]


,regional_fond_product,regional_fond_qty_qx_ha,selected_fond_product,selected_fond_qty_qx_ha,selected_couverture_product,selected_couverture_qty_qx_ha,generic_entry_1_product,generic_entry_1_mode,generic_entry_1_qty_qx_ha,generic_entry_2_product,generic_entry_2_mode,generic_entry_2_qty_qx_ha,generic_entry_3_product,generic_entry_3_mode,generic_entry_3_qty_qx_ha,reco_cost_dh_ha
5097,NPK(10.20.20),3.0,NPK(10.20.20),2.04,Ammonitrates,3.12,NPK(16.11.20),fond,0.72,TSP,fond,0.73,Ammonitrates,couverture,3.39,1024.11
5098,NPK(10.20.20),3.0,NPK(10.20.20),2.55,Ammonitrates,3.95,NPK(16.11.20),fond,0.90,TSP,fond,0.91,Ammonitrates,couverture,4.28,1289.23
5099,NPK(10.20.20),3.0,NPK(10.20.20),3.06,Ammonitrates,4.73,NPK(16.11.20),fond,1.08,TSP,fond,1.10,Ammonitrates,couverture,5.14,1546.27
5100,NPK(10.20.20),3.0,NPK(10.20.20),3.13,Ammonitrates,2.76,NPK(16.11.20),fond,2.24,TSP,fond,0.84,Ammonitrates,couverture,2.62,1378.90
5101,NPK(10.20.20),3.0,NPK(10.20.20),3.86,Ammonitrates,3.52,NPK(16.11.20),fond,2.79,TSP,fond,1.03,Ammonitrates,couverture,3.33,1730.92


In [23]:
print("Selected section entry counts:")
print(parsed_structured["selected_entry_count"].value_counts().sort_index())

print("\nGeneric section entry counts:")
print(parsed_structured["generic_entry_count"].value_counts().sort_index())

print(f"\nMax generic entries captured: {parsed_structured['generic_entry_count'].max()}")

Selected section entry counts:
selected_entry_count
1      4579
2    382621
Name: count, dtype: int64

Generic section entry counts:
generic_entry_count
1     87727
2    265419
3     34054
Name: count, dtype: int64

Max generic entries captured: 3


In [24]:
display_columns = [
    "regional_fond_product",
    "regional_fond_qty_qx_ha",
    "selected_fond_product",
    "selected_fond_qty_qx_ha",
    "selected_couverture_product",
    "selected_couverture_qty_qx_ha",
]

for idx in range(1, max_generic_entries + 1):
    display_columns.extend([
        f"generic_entry_{idx}_product",
        f"generic_entry_{idx}_mode",
        f"generic_entry_{idx}_qty_qx_ha",
    ])

print(grid_df[grid_df["regional_recommendations"].notna()==True][display_columns].iloc[60])

regional_fond_product            NPK(10.20.20)
regional_fond_qty_qx_ha                    3.0
selected_fond_product            NPK(10.20.20)
selected_fond_qty_qx_ha                   3.58
selected_couverture_product       Ammonitrates
selected_couverture_qty_qx_ha             4.34
generic_entry_1_product          NPK(16.11.20)
generic_entry_1_mode                      fond
generic_entry_1_qty_qx_ha                 3.58
generic_entry_2_product                    TSP
generic_entry_2_mode                      fond
generic_entry_2_qty_qx_ha                 0.55
generic_entry_3_product           Ammonitrates
generic_entry_3_mode                couverture
generic_entry_3_qty_qx_ha                 3.69
Name: 5321, dtype: object


In [25]:
print(grid_df[grid_df["regional_recommendations"].notna()==True]["regional_recommendations"].iloc[0])

Recommandations basées sur la formule régionale : 
3qx/ha du NPK(10.20.20) comme engrais de fond 
pour un rendement optimale de la rÃ©gion sÃ©lÃ©ctionnÃ©e
Les recommandations pour le rendement sÃ©lÃ©ctionnÃ©: 
2.04qx/ha du NPK(10.20.20) comme engrais de fond
3.12qx/ha d'Ammonitrates comme engrais de couverture
Recommandations basées sur les formules gÃ©nÃ©riques : 
0.72qx/ha du NPK(16.11.20) comme engrais de fond
0.73qx/ha du TSP comme engrais de fond
3.39qx/ha d'Ammonitrates comme engrais de couverture
pour un cout de 1024.11 dh/ha


## Parsing text data

In [26]:
from IPython.display import display

# Focus on rows that contain parsed generic recommendations
generic_subset = grid_df.loc[non_null_mask].copy()

# Build a tidy/long view of each generic fertilizer entry across slots
generic_long_frames = []
for slot in range(1, max_generic_entries + 1):
    col_map = {
        f"generic_entry_{slot}_product": "product",
        f"generic_entry_{slot}_mode": "mode",
        f"generic_entry_{slot}_qty_qx_ha": "qty_qx_ha",
    }
    slot_df = (
        generic_subset[list(col_map.keys())]
        .rename(columns=col_map)
        .assign(slot=slot)
        .reset_index()
        .rename(columns={"index": "recommendation_index"})
        .dropna(subset=["product"])
    )
    generic_long_frames.append(slot_df)

generic_long = pd.concat(generic_long_frames, ignore_index=True)
total_generic_rows = len(generic_subset)
total_generic_lines = len(generic_long)

print(f"Generic recommendation rows: {total_generic_rows:,}")
print(f"Individual generic fertilizer lines: {total_generic_lines:,}")

# 1) How often does each product/mode combo appear and what are the typical rates?
generic_product_stats = (
    generic_long.groupby(["product", "mode"], as_index=False)
    .agg(
        recommendation_rows=("recommendation_index", "nunique"),
        appearances=("qty_qx_ha", "size"),
        qty_mean=("qty_qx_ha", "mean"),
        qty_median=("qty_qx_ha", "median"),
        qty_std=("qty_qx_ha", "std"),
        qty_min=("qty_qx_ha", "min"),
        qty_max=("qty_qx_ha", "max"),
    )
    .sort_values("recommendation_rows", ascending=False)
)
generic_product_stats["share_pct"] = (
    generic_product_stats["recommendation_rows"] / total_generic_rows
) * 100
print("Top generic fertilizers by share of recommendation rows:")
display(generic_product_stats.head(10))

# 2) Summaries of the entire generic formula per row
qty_columns = [
    f"generic_entry_{idx}_qty_qx_ha" for idx in range(1, max_generic_entries + 1)
]

def build_formula(row, include_qty=False):
    entries = []
    for slot in range(1, max_generic_entries + 1):
        product = row.get(f"generic_entry_{slot}_product")
        mode = row.get(f"generic_entry_{slot}_mode")
        qty = row.get(f"generic_entry_{slot}_qty_qx_ha")
        if pd.isna(product):
            continue
        label = f"{product} ({mode})"
        if include_qty and pd.notna(qty):
            label += f" - {qty:.2f} qx/ha"
        entries.append(label)
    return " + ".join(sorted(entries))

generic_subset = generic_subset.assign(
    generic_formula_products=lambda df: df.apply(
        lambda row: build_formula(row, include_qty=False), axis=1
    ),
    generic_formula_full=lambda df: df.apply(
        lambda row: build_formula(row, include_qty=True), axis=1
    ),
    generic_total_qty_qx_ha=generic_subset[qty_columns].sum(axis=1, skipna=True),
)

generic_formula_summary = (
    generic_subset.groupby("generic_formula_products")
    .agg(
        occurrences=("generic_formula_products", "size"),
        avg_total_qty=("generic_total_qty_qx_ha", "mean"),
        median_total_qty=("generic_total_qty_qx_ha", "median"),
        std_total_qty=("generic_total_qty_qx_ha", "std"),
    )
    .sort_values("occurrences", ascending=False)
    .reset_index()
)
generic_formula_summary["share_pct"] = (
    generic_formula_summary["occurrences"] / total_generic_rows
) * 100
print("Most common generic formula combinations (ignoring exact rates):")
display(generic_formula_summary.head(15))

print("Most common generic formulas with exact rates:")
top_full_formulas = (
    generic_subset["generic_formula_full"]
    .value_counts()
    .head(10)
    .rename_axis("generic_formula_full")
    .reset_index(name="occurrences")
)
display(top_full_formulas)

# 3) Distribution of entry counts and total generic quantities
entry_count_summary = (
    generic_subset.groupby("generic_entry_count")
    .agg(
        rows=("generic_entry_count", "size"),
        avg_total_qty=("generic_total_qty_qx_ha", "mean"),
        median_total_qty=("generic_total_qty_qx_ha", "median"),
    )
    .reset_index()
    .sort_values("generic_entry_count")
)
entry_count_summary["share_pct"] = (
    entry_count_summary["rows"] / total_generic_rows
) * 100
print("Generic entry count distribution and total quantity stats:")
display(entry_count_summary)


Generic recommendation rows: 387,200
Individual generic fertilizer lines: 720,727
Top generic fertilizers by share of recommendation rows:


,product,mode,recommendation_rows,appearances,qty_mean,qty_median,qty_std,qty_min,qty_max,share_pct
0,Ammonitrates,couverture,387141,387141,4.177573,4.40,1.102210,0.01,6.17,99.984762
2,TSP,fond,272905,272905,1.307931,1.39,0.576181,0.01,2.62,70.481663
1,NPK(16.11.20),fond,60681,60681,3.990951,3.85,2.462568,0.01,10.28,15.671746


Most common generic formula combinations (ignoring exact rates):


,generic_formula_products,occurrences,avg_total_qty,median_total_qty,std_total_qty,share_pct
0,Ammonitrates (couverture) + TSP (fond),238848,5.863407,5.91,1.141010,61.685950
1,Ammonitrates (couverture),87671,4.392157,4.47,0.822548,22.642304
2,Ammonitrates (couverture) + NPK(16.11.20) (fon...,34054,7.108979,7.05,1.544703,8.794938
3,Ammonitrates (couverture) + NPK(16.11.20) (fond),26568,7.090160,6.97,1.680110,6.861570
4,NPK(16.11.20) (fond),56,7.438393,7.83,1.502316,0.014463
5,NPK(16.11.20) (fond) + TSP (fond),3,6.970000,7.46,0.874814,0.000775


Most common generic formulas with exact rates:


,generic_formula_full,occurrences
0,Ammonitrates (couverture) - 4.47 qx/ha + TSP (...,13480
1,Ammonitrates (couverture) - 5.41 qx/ha + TSP (...,13440
2,Ammonitrates (couverture) - 3.49 qx/ha + TSP (...,12993
3,Ammonitrates (couverture) - 5.41 qx/ha,9034
4,Ammonitrates (couverture) - 4.47 qx/ha,8948
5,Ammonitrates (couverture) - 5.41 qx/ha + TSP (...,8567
6,Ammonitrates (couverture) - 4.47 qx/ha + TSP (...,8541
7,Ammonitrates (couverture) - 3.49 qx/ha,8456
8,Ammonitrates (couverture) - 3.49 qx/ha + TSP (...,8127
9,Ammonitrates (couverture) - 4.16 qx/ha,4960


Generic entry count distribution and total quantity stats:


,generic_entry_count,rows,avg_total_qty,median_total_qty,share_pct
0,1.0,87727,4.394102,4.47,22.656767
1,2.0,265419,5.986215,6.01,68.548295
2,3.0,34054,7.108979,7.05,8.794938


### Reverse-engineering the generic formula

The following cells treat each parsed recommendation as an input-output example, explore how input features map to generic formula templates, and fit lightweight surrogate models (classification + regression) to approximate the hidden rules.

In [27]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_text
from sklearn.metrics import classification_report, mean_absolute_error, r2_score

# Ensure we work on a fresh copy to avoid accidental mutation in earlier cells
generic_subset = generic_subset.copy()

# Derive a coarse bucket for each generic formula (structure only, no rates)
def categorize_formula(formula: str) -> str:
    if not isinstance(formula, str) or not formula:
        return np.nan
    has_tsp = "TSP" in formula
    has_npk16 = "NPK(16.11.20)" in formula
    if has_tsp and has_npk16:
        return "Ammonitrates + TSP + NPK16"
    if has_tsp:
        return "Ammonitrates + TSP"
    if has_npk16:
        return "Ammonitrates + NPK16"
    return "Ammonitrates only"

if "generic_formula_products" not in generic_subset:
    generic_subset = generic_subset.assign(
        generic_formula_products=generic_subset.apply(
            lambda row: " + ".join(
                sorted(
                    f"{row.get(f'generic_entry_{slot}_product')} ({row.get(f'generic_entry_{slot}_mode')})"
                    for slot in range(1, max_generic_entries + 1)
                    if pd.notna(row.get(f"generic_entry_{slot}_product"))
                )
            ),
            axis=1,
        )
    )

generic_subset["generic_formula_bucket"] = generic_subset["generic_formula_products"].apply(categorize_formula)

# Extract per-product dosage columns for downstream regressions
def extract_product_qty(row, product_name):
    for slot in range(1, max_generic_entries + 1):
        if row.get(f"generic_entry_{slot}_product") == product_name:
            return row.get(f"generic_entry_{slot}_qty_qx_ha")
    return np.nan

product_qty_columns = {
    "Ammonitrates": "generic_qty_ammonitrates",
    "TSP": "generic_qty_tsp",
    "NPK(16.11.20)": "generic_qty_npk_16_11_20",
}

for product, col_name in product_qty_columns.items():
    generic_subset[col_name] = generic_subset.apply(lambda row, prod=product: extract_product_qty(row, prod), axis=1)

default_feature_cols = [
    "ph",
    "organic_matter_pct",
    "p2o5_mgkg",
    "k2o_mgkg",
    "scenario_expected_yield",
    "soil_type",
    "texture",
    "region",
    "province",
    "target_crop_name",
]

model_df = generic_subset.dropna(subset=["generic_formula_bucket"]).copy()
print("Generic formula bucket distribution (% of rows):")
print((model_df["generic_formula_bucket"].value_counts(normalize=True) * 100).round(2))

Generic formula bucket distribution (% of rows):
generic_formula_bucket
Ammonitrates + TSP            61.69
Ammonitrates only             22.64
Ammonitrates + TSP + NPK16     8.80
Ammonitrates + NPK16           6.88
Name: proportion, dtype: float64


In [28]:
# Bin a few agronomic drivers to inspect how they correlate with the formula buckets
model_df = model_df.copy()
model_df["p2o5_bin"] = pd.cut(
    model_df["p2o5_mgkg"],
    bins=[-1, 10, 15, 20, 30, 50, 200],
    labels=["<=10", "10-15", "15-20", "20-30", "30-50", ">50"],
    include_lowest=True,
)
model_df["k2o_bin"] = pd.cut(
    model_df["k2o_mgkg"],
    bins=[-1, 50, 100, 150, 200, 300, 600],
    labels=["<=50", "50-100", "100-150", "150-200", "200-300", ">300"],
    include_lowest=True,
)
model_df["scenario_yield_bin"] = pd.cut(
    model_df["scenario_expected_yield"],
    bins=[0, 20, 30, 40, 50, 60, 90],
    labels=["<=20", "20-30", "30-40", "40-50", "50-60", ">60"],
    include_lowest=True,
)


def share_table(feature):
    table = (
        model_df.groupby([feature, "generic_formula_bucket"])
        .size()
        .unstack(fill_value=0)
        .pipe(lambda df: (df.T / df.sum(axis=1)).T * 100)
        .round(1)
    )
    print(f"Share of formula buckets by {feature}:")
    display(table)

share_table("p2o5_bin")
share_table("k2o_bin")
share_table("scenario_yield_bin")

Share of formula buckets by p2o5_bin:


/var/folders/vg/p5dpncwj1nl960cnf3wq8_900000gn/T/ipykernel_76841/829053979.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  model_df.groupby([feature, "generic_formula_bucket"])


generic_formula_bucket,Ammonitrates + NPK16,Ammonitrates + TSP,Ammonitrates + TSP + NPK16,Ammonitrates only
p2o5_bin,,,,
<=10,0.6,65.0,34.5,0.0
10-15,0.7,89.7,9.6,0.0
15-20,2.3,79.9,17.8,0.0
20-30,6.9,85.0,8.1,0.0
30-50,21.0,34.8,1.6,42.6
>50,8.9,0.0,0.0,91.1


Share of formula buckets by k2o_bin:


/var/folders/vg/p5dpncwj1nl960cnf3wq8_900000gn/T/ipykernel_76841/829053979.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  model_df.groupby([feature, "generic_formula_bucket"])


generic_formula_bucket,Ammonitrates + NPK16,Ammonitrates + TSP,Ammonitrates + TSP + NPK16,Ammonitrates only
k2o_bin,,,,
<=50,35.5,0.0,64.5,0.0
50-100,72.0,0.0,28.0,0.0
100-150,50.6,0.0,49.4,0.0
150-200,11.8,33.8,30.1,24.3
200-300,0.0,72.7,0.0,27.3
>300,0.0,78.8,0.0,21.2


Share of formula buckets by scenario_yield_bin:


/var/folders/vg/p5dpncwj1nl960cnf3wq8_900000gn/T/ipykernel_76841/829053979.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  model_df.groupby([feature, "generic_formula_bucket"])


generic_formula_bucket,Ammonitrates + NPK16,Ammonitrates + TSP,Ammonitrates + TSP + NPK16,Ammonitrates only
scenario_yield_bin,,,,
<=20,NaN,NaN,NaN,NaN
20-30,NaN,NaN,NaN,NaN
30-40,7.1,61.7,8.5,22.7
40-50,6.8,61.7,8.9,22.6
50-60,6.7,61.7,9.0,22.6
>60,NaN,NaN,NaN,NaN


In [29]:
# Train a lightweight classifier to approximate which bucket applies to each row
numeric_features = [
    "ph",
    "organic_matter_pct",
    "p2o5_mgkg",
    "k2o_mgkg",
    "scenario_expected_yield",
]
categorical_features = [
    "soil_type",
    "texture",
    "region",
    "province",
    "target_crop_name",
]

X = model_df[numeric_features + categorical_features]
y = model_df["generic_formula_bucket"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median"))]
)
categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

clf_pipeline = Pipeline(
    steps=[
        ("preprocess", preprocess),
        (
            "clf",
            DecisionTreeClassifier(
                max_depth=4,
                class_weight="balanced",
                random_state=42,
            ),
        ),
    ]
)

clf_pipeline.fit(X_train, y_train)
y_pred = clf_pipeline.predict(X_test)

print("Classification report (hold-out set):")
print(classification_report(y_test, y_pred))

feature_names = clf_pipeline.named_steps["preprocess"].get_feature_names_out()
tree_rules = export_text(clf_pipeline.named_steps["clf"], feature_names=list(feature_names))
print("Rule-of-thumb decision tree:")
print(tree_rules)


/Users/othmaneechchabi/anaconda3/envs/clean-py312/lib/python3.12/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['region' 'province']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(
/Users/othmaneechchabi/anaconda3/envs/clean-py312/lib/python3.12/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['region' 'province']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(


Classification report (hold-out set):
                            precision    recall  f1-score   support

      Ammonitrates + NPK16       0.95      0.91      0.93      5325
        Ammonitrates + TSP       1.00      1.00      1.00     47770
Ammonitrates + TSP + NPK16       0.93      0.97      0.95      6811
         Ammonitrates only       1.00      1.00      1.00     17534

                  accuracy                           0.99     77440
                 macro avg       0.97      0.97      0.97     77440
              weighted avg       0.99      0.99      0.99     77440

Rule-of-thumb decision tree:
|--- num__k2o_mgkg <= 199.85
|   |--- num__p2o5_mgkg <= 24.35
|   |   |--- num__k2o_mgkg <= 123.35
|   |   |   |--- num__p2o5_mgkg <= 18.85
|   |   |   |   |--- class: Ammonitrates + TSP + NPK16
|   |   |   |--- num__p2o5_mgkg >  18.85
|   |   |   |   |--- class: Ammonitrates + NPK16
|   |   |--- num__k2o_mgkg >  123.35
|   |   |   |--- num__p2o5_mgkg <= 22.75
|   |   |   |   |--- cl

In [30]:
# Fit separate regressors for the dosage of each product
regression_results = []
generic_regressors = {}
reg_numeric = numeric_features
reg_categorical = categorical_features

for product, target_col in product_qty_columns.items():
    subset = model_df.dropna(subset=[target_col])
    if subset.empty:
        continue
    X = subset[reg_numeric + reg_categorical]
    y = subset[target_col]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    reg_pipeline = Pipeline(
        steps=[
            (
                "preprocess",
                ColumnTransformer(
                    transformers=[
                        ("num", Pipeline([("imputer", SimpleImputer(strategy="median"))]), reg_numeric),
                        (
                            "cat",
                            Pipeline(
                                [
                                    ("imputer", SimpleImputer(strategy="most_frequent")),
                                    ("onehot", OneHotEncoder(handle_unknown="ignore")),
                                ]
                            ),
                            reg_categorical,
                        ),
                    ]
                ),
            ),
            ("reg", DecisionTreeRegressor(max_depth=5, random_state=42)),
        ]
    )

    reg_pipeline.fit(X_train, y_train)
    y_pred = reg_pipeline.predict(X_test)

    generic_regressors[product] = reg_pipeline

    regression_results.append(
        {
            "product": product,
            "rows": len(subset),
            "mae": mean_absolute_error(y_test, y_pred),
            "r2": r2_score(y_test, y_pred),
        }
    )

results_df = pd.DataFrame(regression_results)
print("Dosage surrogate performance (DecisionTreeRegressor, depth=5):")
display(results_df)


/Users/othmaneechchabi/anaconda3/envs/clean-py312/lib/python3.12/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['region' 'province']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(
/Users/othmaneechchabi/anaconda3/envs/clean-py312/lib/python3.12/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['region' 'province']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(
/Users/othmaneechchabi/anaconda3/envs/clean-py312/lib/python3.12/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['region' 'province']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(
/Users/othmaneechchabi/anaconda3/envs/clean-py312/lib/python3.12/site-packages/sklearn/impute/_base.py:653: UserWar

Dosage surrogate performance (DecisionTreeRegressor, depth=5):


/Users/othmaneechchabi/anaconda3/envs/clean-py312/lib/python3.12/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['region' 'province']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(


,product,rows,mae,r2
0,Ammonitrates,387141,0.123007,0.976740
1,TSP,272905,0.085682,0.955824
2,NPK(16.11.20),60681,0.185720,0.991445


In [31]:
from pprint import pprint

# Helper to run the surrogate models on arbitrary feature sets
def predict_generic_recommendation(sample_features):
    if "clf_pipeline" not in globals():
        raise RuntimeError("Run the generic classification cell before calling this helper.")
    if "generic_regressors" not in globals() or not generic_regressors:
        raise RuntimeError("Run the dosage regression cell to populate generic_regressors.")

    required_cols = numeric_features + categorical_features
    row = {col: sample_features.get(col, np.nan) for col in required_cols}
    input_df = pd.DataFrame([row])

    formula_bucket = clf_pipeline.predict(input_df)[0]
    bucket_probabilities = dict(zip(clf_pipeline.classes_, clf_pipeline.predict_proba(input_df)[0]))

    dosage_predictions = {}
    for product, model in generic_regressors.items():
        dosage_predictions[product] = float(model.predict(input_df)[0])

    return {
        "formula_bucket": formula_bucket,
        "bucket_probabilities": {k: round(v, 4) for k, v in bucket_probabilities.items()},
        "dosage_predictions_qx_ha": {k: round(v, 2) for k, v in dosage_predictions.items()},
    }

# Example inference: reuse the first parsed row as a feature template
example_index = generic_subset.index[0]
example_features = (
    grid_df.loc[example_index, numeric_features + categorical_features]
    .to_dict()
)

print(f"Example recommendation index: {example_index}")
pprint(predict_generic_recommendation(example_features))


Example recommendation index: 5097
{'bucket_probabilities': {'Ammonitrates + NPK16': np.float64(0.1479),
                          'Ammonitrates + TSP': np.float64(0.0),
                          'Ammonitrates + TSP + NPK16': np.float64(0.8521),
                          'Ammonitrates only': np.float64(0.0)},
 'dosage_predictions_qx_ha': {'Ammonitrates': 3.53,
                              'NPK(16.11.20)': 0.89,
                              'TSP': 0.8},
 'formula_bucket': 'Ammonitrates + TSP + NPK16'}


/Users/othmaneechchabi/anaconda3/envs/clean-py312/lib/python3.12/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['region' 'province']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(
/Users/othmaneechchabi/anaconda3/envs/clean-py312/lib/python3.12/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['region' 'province']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(
/Users/othmaneechchabi/anaconda3/envs/clean-py312/lib/python3.12/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['region' 'province']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(
/Users/othmaneechchabi/anaconda3/envs/clean-py312/lib/python3.12/site-packages/sklearn/impute/_base.py:653: UserWar

In [32]:
generic_subset

,lat,lon,region,province,province_id,commune,soil_type,texture,ph,organic_matter_pct,...,selected_entry_count,generic_entry_count,reco_cost_dh_ha,generic_formula_products,generic_formula_full,generic_total_qty_qx_ha,generic_formula_bucket,generic_qty_ammonitrates,generic_qty_tsp,generic_qty_npk_16_11_20
5097,30.791667,-9.700000,NaN,NaN,30.0,Sidi H\'Mad Ou M\'Barek,Texture globale,globale,8.30,1.32,...,2.0,3.0,1024.11,Ammonitrates (couverture) + NPK(16.11.20) (fon...,Ammonitrates (couverture) - 3.39 qx/ha + NPK(1...,4.84,Ammonitrates + TSP + NPK16,3.39,0.73,0.72
5098,30.791667,-9.700000,NaN,NaN,30.0,Sidi H\'Mad Ou M\'Barek,Texture globale,globale,8.30,1.32,...,2.0,3.0,1289.23,Ammonitrates (couverture) + NPK(16.11.20) (fon...,Ammonitrates (couverture) - 4.28 qx/ha + NPK(1...,6.09,Ammonitrates + TSP + NPK16,4.28,0.91,0.90
5099,30.791667,-9.700000,NaN,NaN,30.0,Sidi H\'Mad Ou M\'Barek,Texture globale,globale,8.30,1.32,...,2.0,3.0,1546.27,Ammonitrates (couverture) + NPK(16.11.20) (fon...,Ammonitrates (couverture) - 5.14 qx/ha + NPK(1...,7.32,Ammonitrates + TSP + NPK16,5.14,1.10,1.08
5100,30.791667,-9.683333,NaN,NaN,30.0,Sidi H\'Mad Ou M\'Barek,Texture globale,globale,8.24,1.40,...,2.0,3.0,1378.90,Ammonitrates (couverture) + NPK(16.11.20) (fon...,Ammonitrates (couverture) - 2.62 qx/ha + NPK(1...,5.70,Ammonitrates + TSP + NPK16,2.62,0.84,2.24
5101,30.791667,-9.683333,NaN,NaN,30.0,Sidi H\'Mad Ou M\'Barek,Texture globale,globale,8.24,1.40,...,2.0,3.0,1730.92,Ammonitrates (couverture) + NPK(16.11.20) (fon...,Ammonitrates (couverture) - 3.33 qx/ha + NPK(1...,7.15,Ammonitrates + TSP + NPK16,3.33,1.03,2.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606120,35.433333,-2.966667,NaN,NaN,37.0,Bni Chiker,Texture globale,globale,8.00,2.00,...,2.0,2.0,920.27,Ammonitrates (couverture) + TSP (fond),Ammonitrates (couverture) - 3.49 qx/ha + TSP (...,4.90,Ammonitrates + TSP,3.49,1.41,NaN
606121,35.433333,-2.966667,NaN,NaN,37.0,Bni Chiker,Texture globale,globale,8.00,2.00,...,2.0,2.0,1167.40,Ammonitrates (couverture) + TSP (fond),Ammonitrates (couverture) - 4.47 qx/ha + TSP (...,6.21,Ammonitrates + TSP,4.47,1.74,NaN
606123,35.433333,-2.958333,NaN,NaN,37.0,Bni Chiker,Texture globale,globale,8.00,2.00,...,2.0,2.0,920.27,Ammonitrates (couverture) + TSP (fond),Ammonitrates (couverture) - 3.49 qx/ha + TSP (...,4.90,Ammonitrates + TSP,3.49,1.41,NaN
606124,35.433333,-2.958333,NaN,NaN,37.0,Bni Chiker,Texture globale,globale,8.00,2.00,...,2.0,2.0,1167.40,Ammonitrates (couverture) + TSP (fond),Ammonitrates (couverture) - 4.47 qx/ha + TSP (...,6.21,Ammonitrates + TSP,4.47,1.74,NaN


### Nutrient budget alignment

Sanity-check that the published N/P/K recommendations (`rec_*_kg_ha`) are linear combinations of the fertilizer lines we parsed (regional + selected + generic).

In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from collections import Counter

FERTILIZER_PRODUCT_LIST = [
    "Ammonitrates",
    "NPK(10.20.20)",
    "NPK(15.30.10)",
    "NPK(9.23.30)",
    "NPK(16.11.20)",
    "TSP",
]

npk_mask = (
    non_null_mask
    & grid_df[["rec_n_kg_ha", "rec_p_kg_ha", "rec_k_kg_ha"]].notna().all(axis=1)
)
npk_calibration_df = grid_df.loc[npk_mask].copy()


def total_qty_for_product(row, product):
    total = 0.0

    def add_if_match(prod, qty):
        if prod == product and pd.notna(qty):
            return qty
        return 0.0

    total += add_if_match(row.get("regional_fond_product"), row.get("regional_fond_qty_qx_ha"))
    total += add_if_match(row.get("selected_fond_product"), row.get("selected_fond_qty_qx_ha"))
    total += add_if_match(row.get("selected_couverture_product"), row.get("selected_couverture_qty_qx_ha"))

    for slot in range(1, max_generic_entries + 1):
        total += add_if_match(
            row.get(f"generic_entry_{slot}_product"),
            row.get(f"generic_entry_{slot}_qty_qx_ha"),
        )

    return total


for product in FERTILIZER_PRODUCT_LIST:
    npk_calibration_df[f"qty_{product}_qx_ha"] = npk_calibration_df.apply(
        lambda row, prod=product: total_qty_for_product(row, prod), axis=1
    )

feature_cols = [f"qty_{prod}_qx_ha" for prod in FERTILIZER_PRODUCT_LIST]
X = npk_calibration_df[feature_cols].values

nutrient_targets = {
    "N": "rec_n_kg_ha",
    "P": "rec_p_kg_ha",
    "K": "rec_k_kg_ha",
}

calibrated_models = {}
metrics_rows = []

for nutrient, target_col in nutrient_targets.items():
    y = npk_calibration_df[target_col].values
    linreg = LinearRegression(fit_intercept=False)
    linreg.fit(X, y)
    preds = linreg.predict(X)
    npk_calibration_df[f"pred_{nutrient}_kg_ha"] = preds

    diff = preds - y
    metrics_rows.append(
        {
            "nutrient": nutrient,
            "rows": len(y),
            "mae": mean_absolute_error(y, preds),
            "rmse": float((diff ** 2).mean() ** 0.5),
            "bias": float(diff.mean()),
            "r2": r2_score(y, preds),
        }
    )

    calibrated_models[nutrient] = linreg

metrics_df = pd.DataFrame(metrics_rows)
print("Data-driven nutrient reconstruction (kg/ha):")
display(metrics_df)

coeff_rows = []
for nutrient, model in calibrated_models.items():
    for product, coef in zip(FERTILIZER_PRODUCT_LIST, model.coef_):
        coeff_rows.append(
            {
                "product": product,
                "nutrient": nutrient,
                "kg_per_qx": coef,
                "pct_of_product_mass": coef / 100.0,
            }
        )

coeff_df = pd.DataFrame(coeff_rows)
print("Estimated kg of nutrient delivered per qx/ha of each fertilizer (data-driven):")
display(coeff_df.pivot(index="product", columns="nutrient", values="kg_per_qx"))

print("Sample reconstructions (first 5 rows):")
display(
    npk_calibration_df[
        [
            "rec_n_kg_ha",
            "pred_N_kg_ha",
            "rec_p_kg_ha",
            "pred_P_kg_ha",
            "rec_k_kg_ha",
            "pred_K_kg_ha",
        ]
    ].head()
)


Data-driven nutrient reconstruction (kg/ha):


,nutrient,rows,mae,rmse,bias,r2
0,N,387200,1.431833,2.031956,0.227468,0.99433
1,P,387200,3.262006,8.346537,0.034772,0.94237
2,K,387200,0.008079,0.023006,-0.000063,1.00000


Estimated kg of nutrient delivered per qx/ha of each fertilizer (data-driven):


nutrient,K,N,P
product,,,
Ammonitrates,-0.000062,15.649301,-0.243807
NPK(10.20.20),0.000192,2.641049,0.369711
NPK(15.30.10),0.000192,4.215353,0.783363
NPK(16.11.20),19.999206,9.441907,6.668296
NPK(9.23.30),0.000229,2.105224,0.236065
TSP,-0.000248,4.460211,45.010665


Sample reconstructions (first 5 rows):


,rec_n_kg_ha,pred_N_kg_ha,rec_p_kg_ha,pred_P_kg_ha,rec_k_kg_ha,pred_K_kg_ha
5097,123.33,125.241963,40.74,37.935117,14.34,14.399811
5098,155.66,156.008077,50.93,47.006533,17.92,17.999615
5099,186.66,185.566849,61.11,56.547562,21.50,21.599421
5100,122.32,125.279318,62.50,53.700587,44.71,44.798856
5101,154.65,156.252245,77.17,65.831668,55.89,55.798421


### Regional & selected-yield recommendations

Extend the reverse-engineering workflow to the regional baseline formula and the yield-specific recommendations to see how soil / target attributes drive those prescriptions.

In [34]:

# Prepare datasets for regional and selected sections
regional_selected_df = grid_df.loc[non_null_mask].copy()

regional_selected_df["regional_formula_product"] = regional_selected_df["regional_fond_product"].fillna("Unknown")
regional_selected_df["regional_formula_qty_qx_ha"] = regional_selected_df["regional_fond_qty_qx_ha"]

# Combination string for selected-yield recommendations
regional_selected_df["selected_combo_bucket"] = regional_selected_df.apply(
    lambda row: f"{row['selected_fond_product'] or 'None'} + {row['selected_couverture_product'] or 'None'}",
    axis=1,
)
regional_selected_df["selected_has_fond"] = regional_selected_df["selected_fond_product"].notna()
regional_selected_df["selected_has_couverture"] = regional_selected_df["selected_couverture_product"].notna()

print("Regional formula distribution (%):")
print((regional_selected_df["regional_formula_product"].value_counts(normalize=True) * 100).round(2))

print("Selected-yield combo distribution (% of rows with recommendations):")
print((regional_selected_df["selected_combo_bucket"].value_counts(normalize=True) * 100).round(2).head(10))

Regional formula distribution (%):
regional_formula_product
NPK(10.20.20)    61.66
NPK(15.30.10)    26.43
NPK(9.23.30)     11.91
Name: proportion, dtype: float64
Selected-yield combo distribution (% of rows with recommendations):
selected_combo_bucket
NPK(10.20.20) + Ammonitrates    61.66
NPK(15.30.10) + Ammonitrates    25.25
NPK(9.23.30) + Ammonitrates     11.91
NPK(15.30.10) + None             1.18
Name: proportion, dtype: float64


In [35]:
# Share tables for regional product and selected combo versus agronomic bins
reg_sel_df = regional_selected_df.copy()
reg_sel_df["p2o5_bin"] = pd.cut(
    reg_sel_df["p2o5_mgkg"],
    bins=[-1, 10, 15, 20, 30, 50, 200],
    labels=["<=10", "10-15", "15-20", "20-30", "30-50", ">50"],
    include_lowest=True,
)
reg_sel_df["k2o_bin"] = pd.cut(
    reg_sel_df["k2o_mgkg"],
    bins=[-1, 50, 100, 150, 200, 300, 600],
    labels=["<=50", "50-100", "100-150", "150-200", "200-300", ">300"],
    include_lowest=True,
)
reg_sel_df["scenario_yield_bin"] = pd.cut(
    reg_sel_df["scenario_expected_yield"],
    bins=[0, 20, 30, 40, 50, 60, 90],
    labels=["<=20", "20-30", "30-40", "40-50", "50-60", ">60"],
    include_lowest=True,
)


def share_table(df, feature, target):
    table = (
        df.groupby([feature, target]).size().unstack(fill_value=0)
        .pipe(lambda data: (data.T / data.sum(axis=1)).T * 100)
        .round(1)
    )
    print(f"Share of {target} by {feature}:")
    display(table)

share_table(reg_sel_df, "p2o5_bin", "regional_formula_product")
share_table(reg_sel_df, "k2o_bin", "regional_formula_product")
share_table(reg_sel_df, "scenario_yield_bin", "regional_formula_product")

share_table(reg_sel_df, "p2o5_bin", "selected_combo_bucket")
share_table(reg_sel_df, "k2o_bin", "selected_combo_bucket")
share_table(reg_sel_df, "scenario_yield_bin", "selected_combo_bucket")

Share of regional_formula_product by p2o5_bin:


/var/folders/vg/p5dpncwj1nl960cnf3wq8_900000gn/T/ipykernel_76841/3778291039.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby([feature, target]).size().unstack(fill_value=0)


regional_formula_product,NPK(10.20.20),NPK(15.30.10),NPK(9.23.30)
p2o5_bin,,,
<=10,60.2,18.3,21.4
10-15,53.3,40.0,6.8
15-20,55.5,30.9,13.6
20-30,64.7,22.1,13.2
30-50,67.7,16.9,15.5
>50,68.0,22.5,9.6


Share of regional_formula_product by k2o_bin:


/var/folders/vg/p5dpncwj1nl960cnf3wq8_900000gn/T/ipykernel_76841/3778291039.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby([feature, target]).size().unstack(fill_value=0)


...

/var/folders/vg/p5dpncwj1nl960cnf3wq8_900000gn/T/ipykernel_76841/3778291039.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby([feature, target]).size().unstack(fill_value=0)


selected_combo_bucket,NPK(10.20.20) + Ammonitrates,NPK(15.30.10) + Ammonitrates,NPK(15.30.10) + None,NPK(9.23.30) + Ammonitrates
k2o_bin,,,,
<=50,6.6,0.0,29.8,63.6
50-100,18.3,0.0,20.6,61.1
100-150,49.3,4.7,7.3,38.6
150-200,74.3,11.2,0.0,14.5
200-300,72.8,17.6,0.0,9.6
>300,62.5,30.5,0.0,7.0


Share of selected_combo_bucket by scenario_yield_bin:


/var/folders/vg/p5dpncwj1nl960cnf3wq8_900000gn/T/ipykernel_76841/3778291039.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby([feature, target]).size().unstack(fill_value=0)


selected_combo_bucket,NPK(10.20.20) + Ammonitrates,NPK(15.30.10) + Ammonitrates,NPK(15.30.10) + None,NPK(9.23.30) + Ammonitrates
scenario_yield_bin,,,,
<=20,NaN,NaN,NaN,NaN
20-30,NaN,NaN,NaN,NaN
30-40,61.5,25.5,1.2,11.8
40-50,61.7,25.2,1.2,11.9
50-60,61.8,25.1,1.2,12.0
>60,NaN,NaN,NaN,NaN


#### Regional surrogate models

Decision-tree classifier/regressor with lat/lon-enhanced features to mimic the regional baseline formula.

In [36]:

regional_model_df = reg_sel_df.dropna(subset=["regional_formula_product"]).copy()

regional_numeric_features = numeric_features + ["lat", "lon"]
regional_categorical_features = categorical_features

X_regional = regional_model_df[regional_numeric_features + regional_categorical_features]
y_regional = regional_model_df["regional_formula_product"]

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_regional, y_regional, test_size=0.2, random_state=42, stratify=y_regional
)

regional_preprocess = ColumnTransformer(
    transformers=[
        ("num", Pipeline([("imputer", SimpleImputer(strategy="median"))]), regional_numeric_features),
        (
            "cat",
            Pipeline(
                [
                    ("imputer", SimpleImputer(strategy="most_frequent")),
                    ("onehot", OneHotEncoder(handle_unknown="ignore")),
                ]
            ),
            regional_categorical_features,
        ),
    ]
)

regional_clf = Pipeline(
    steps=[
        ("preprocess", regional_preprocess),
        ("clf", DecisionTreeClassifier(max_depth=10, class_weight="balanced", random_state=42)),
    ]
)

regional_clf.fit(X_train_reg, y_train_reg)
y_reg_pred = regional_clf.predict(X_test_reg)

print("Regional formula classifier (lat/lon features):")
print(classification_report(y_test_reg, y_reg_pred))

reg_feature_names = regional_clf.named_steps["preprocess"].get_feature_names_out()
print(export_text(regional_clf.named_steps["clf"], feature_names=list(reg_feature_names)))

regional_qty_df = regional_model_df.dropna(subset=["regional_formula_qty_qx_ha"])
if not regional_qty_df.empty:
    X_qty = regional_qty_df[regional_numeric_features + regional_categorical_features]
    y_qty = regional_qty_df["regional_formula_qty_qx_ha"]

    X_train_qty, X_test_qty, y_train_qty, y_test_qty = train_test_split(
        X_qty, y_qty, test_size=0.2, random_state=42
    )

    regional_reg_preprocess = ColumnTransformer(
        transformers=[
            ("num", Pipeline([("imputer", SimpleImputer(strategy="median"))]), regional_numeric_features),
            (
                "cat",
                Pipeline(
                    [
                        ("imputer", SimpleImputer(strategy="most_frequent")),
                        ("onehot", OneHotEncoder(handle_unknown="ignore")),
                    ]
                ),
                regional_categorical_features,
            ),
        ]
    )

    regional_regressor = Pipeline(
        steps=[
            ("preprocess", regional_reg_preprocess),
            ("reg", DecisionTreeRegressor(max_depth=12, random_state=42)),
        ]
    )

    regional_regressor.fit(X_train_qty, y_train_qty)
    y_qty_pred = regional_regressor.predict(X_test_qty)

    print("Regional fond dosage surrogate:")
    print(
        {
            "mae": round(mean_absolute_error(y_test_qty, y_qty_pred), 3),
            "r2": round(r2_score(y_test_qty, y_qty_pred), 3),
            "rows": len(regional_qty_df),
        }
    )
else:
    print("No regional dosage rows available for regression.")


/Users/othmaneechchabi/anaconda3/envs/clean-py312/lib/python3.12/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['region' 'province']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(
/Users/othmaneechchabi/anaconda3/envs/clean-py312/lib/python3.12/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['region' 'province']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(


Regional formula classifier (lat/lon features):
               precision    recall  f1-score   support

NPK(10.20.20)       1.00      0.99      0.99     47746
NPK(15.30.10)       0.98      0.99      0.99     20468
 NPK(9.23.30)       0.97      1.00      0.98      9226

     accuracy                           0.99     77440
    macro avg       0.98      0.99      0.99     77440
 weighted avg       0.99      0.99      0.99     77440

|--- num__lon <= -5.77
|   |--- num__lat <= 33.22
|   |   |--- num__ph <= 6.63
|   |   |   |--- num__lat <= 33.19
|   |   |   |   |--- class: NPK(10.20.20)
|   |   |   |--- num__lat >  33.19
|   |   |   |   |--- num__lon <= -6.64
|   |   |   |   |   |--- class: NPK(9.23.30)
|   |   |   |   |--- num__lon >  -6.64
|   |   |   |   |   |--- class: NPK(9.23.30)
|   |   |--- num__ph >  6.63
|   |   |   |--- num__ph <= 7.08
|   |   |   |   |--- num__organic_matter_pct <= 2.51
|   |   |   |   |   |--- num__p2o5_mgkg <= 131.15
|   |   |   |   |   |   |--- class: NPK(

/Users/othmaneechchabi/anaconda3/envs/clean-py312/lib/python3.12/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['region' 'province']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(


Regional fond dosage surrogate:
{'mae': 0.009, 'r2': 0.976, 'rows': 387200}


/Users/othmaneechchabi/anaconda3/envs/clean-py312/lib/python3.12/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['region' 'province']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(


#### Selected-yield surrogate models

Separate classifier/regressors for the selected-yield combo and its fond/couverture rates (lat/lon included).

In [37]:

selected_model_df = reg_sel_df.dropna(subset=["selected_combo_bucket"]).copy()

selected_numeric_features = numeric_features + ["lat", "lon"]
selected_categorical_features = categorical_features

X_sel = selected_model_df[selected_numeric_features + selected_categorical_features]
y_sel = selected_model_df["selected_combo_bucket"]

X_train_sel, X_test_sel, y_train_sel, y_test_sel = train_test_split(
    X_sel, y_sel, test_size=0.2, random_state=42, stratify=y_sel
)

selected_preprocess = ColumnTransformer(
    transformers=[
        ("num", Pipeline([("imputer", SimpleImputer(strategy="median"))]), selected_numeric_features),
        (
            "cat",
            Pipeline(
                [
                    ("imputer", SimpleImputer(strategy="most_frequent")),
                    ("onehot", OneHotEncoder(handle_unknown="ignore")),
                ]
            ),
            selected_categorical_features,
        ),
    ]
)

selected_clf = Pipeline(
    steps=[
        ("preprocess", selected_preprocess),
        ("clf", DecisionTreeClassifier(max_depth=10, class_weight="balanced", random_state=42)),
    ]
)

selected_clf.fit(X_train_sel, y_train_sel)
y_sel_pred = selected_clf.predict(X_test_sel)

print("Selected combo classifier (lat/lon features):")
print(classification_report(y_test_sel, y_sel_pred))

sel_feature_names = selected_clf.named_steps["preprocess"].get_feature_names_out()
print(export_text(selected_clf.named_steps["clf"], feature_names=list(sel_feature_names)))

selected_regression_targets = {
    "Selected fond qty": "selected_fond_qty_qx_ha",
    "Selected couverture qty": "selected_couverture_qty_qx_ha",
}

selected_regression_rows = []
for label, target_col in selected_regression_targets.items():
    subset = selected_model_df.dropna(subset=[target_col])
    if subset.empty:
        continue
    X_target = subset[selected_numeric_features + selected_categorical_features]
    y_target = subset[target_col]

    X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(
        X_target, y_target, test_size=0.2, random_state=42
    )

    selected_reg_preprocess = ColumnTransformer(
        transformers=[
            ("num", Pipeline([("imputer", SimpleImputer(strategy="median"))]), selected_numeric_features),
            (
                "cat",
                Pipeline(
                    [
                        ("imputer", SimpleImputer(strategy="most_frequent")),
                        ("onehot", OneHotEncoder(handle_unknown="ignore")),
                    ]
                ),
                selected_categorical_features,
            ),
        ]
    )

    reg_model = Pipeline(
        steps=[
            ("preprocess", selected_reg_preprocess),
            ("reg", DecisionTreeRegressor(max_depth=12, random_state=42)),
        ]
    )

    reg_model.fit(X_train_t, y_train_t)
    y_pred_t = reg_model.predict(X_test_t)

    selected_regression_rows.append(
        {
            "target": label,
            "rows": len(subset),
            "mae": mean_absolute_error(y_test_t, y_pred_t),
            "r2": r2_score(y_test_t, y_pred_t),
        }
    )

if selected_regression_rows:
    print("Selected dosage surrogate performance:")
    display(pd.DataFrame(selected_regression_rows))
else:
    print("No selected dosage rows available for regression.")


/Users/othmaneechchabi/anaconda3/envs/clean-py312/lib/python3.12/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['region' 'province']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(
/Users/othmaneechchabi/anaconda3/envs/clean-py312/lib/python3.12/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['region' 'province']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(


Selected combo classifier (lat/lon features):
                              precision    recall  f1-score   support

NPK(10.20.20) + Ammonitrates       1.00      0.98      0.99     47746
NPK(15.30.10) + Ammonitrates       0.96      0.99      0.97     19552
        NPK(15.30.10) + None       0.94      1.00      0.97       916
 NPK(9.23.30) + Ammonitrates       0.97      0.99      0.98      9226

                    accuracy                           0.98     77440
                   macro avg       0.97      0.99      0.98     77440
                weighted avg       0.98      0.98      0.98     77440

|--- num__k2o_mgkg <= 131.45
|   |--- num__lon <= -6.28
|   |   |--- num__lon <= -7.04
|   |   |   |--- class: NPK(10.20.20) + Ammonitrates
|   |   |--- num__lon >  -7.04
|   |   |   |--- num__lon <= -6.31
|   |   |   |   |--- num__lat <= 32.93
|   |   |   |   |   |--- class: NPK(10.20.20) + Ammonitrates
|   |   |   |   |--- num__lat >  32.93
|   |   |   |   |   |--- num__lon <= -6.32
|  

/Users/othmaneechchabi/anaconda3/envs/clean-py312/lib/python3.12/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['region' 'province']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(
/Users/othmaneechchabi/anaconda3/envs/clean-py312/lib/python3.12/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['region' 'province']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(
/Users/othmaneechchabi/anaconda3/envs/clean-py312/lib/python3.12/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['region' 'province']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(


Selected dosage surrogate performance:


/Users/othmaneechchabi/anaconda3/envs/clean-py312/lib/python3.12/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['region' 'province']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(


,target,rows,mae,r2
0,Selected fond qty,387200,0.091890,0.976939
1,Selected couverture qty,382621,0.040064,0.984098


### RandomForest surrogate training (offline Fertimap)
Train a light RandomForest surrogate on `fertimap_grid.csv` and persist the models for the FastAPI backend.


In [ ]:
from pathlib import Path
import sys

# Resolve repo root so we can import the server module from the notebook
cwd = Path.cwd()
if (cwd / 'fertimap_grid.csv').exists():
    data_dir = cwd
elif (cwd / 'public' / 'data' / 'fertimap_grid.csv').exists():
    data_dir = cwd / 'public' / 'data'
elif (cwd.parent / 'public' / 'data' / 'fertimap_grid.csv').exists():
    data_dir = cwd.parent / 'public' / 'data'
else:
    raise FileNotFoundError('Could not find fertimap_grid.csv next to this notebook')

repo_root = data_dir.parent.parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from server.fertimap_service import MODEL_PATH, TRAIN_SAMPLE_SIZE, train_local_models

bundle = train_local_models()
metadata = bundle.get('metadata', {})
print(f"Saved RF surrogate models to {MODEL_PATH}")
print(f"Training rows: {bundle.get('training_rows')} (sampled: {metadata.get('sampled', False)})")
